# Практическая работа


### Цель практической работы
В данном модуле вы рассмотрели работу с публичным API и Apache Kafka и попрактиковались в ней, но сделали это по отдельности: из API выгрузили данные и проанализировали их в Jupyter Notebook, а с Kafka генерировали сообщения искусственно и отправляли их. В реальных задачах нередко результат работы одного сервиса используется как данные на вход для другого. 

Представим ситуацию, когда есть поток событий, получаемых из API, который необходимо обрабатывать в реальном времени и результаты которого нужно периодически сохранять. При работе с API сайта Coingecko вы получали запросом список всех coins и историю их цен в BTC и USD. Помимо этого, API позволяет получать данные об объёме торгов (volume) на разных биржах (exchanges). 

Реализуем обработку потока данных об объёме торгов на разных биржах, чтобы закрепить API и Kafka.

In [240]:
import requests
import json
import socket
from time import sleep

from confluent_kafka import Producer


Считайте список всех бирж из API и получите информацию об объёмах торгов за один последний день для любой из бирж. Обратите внимание на гранулярность данных в случае выгрузки одного дня и более двух дней.

In [241]:
import requests
import json

# Получаем список всех бирж
exchanges_url = "https://api.coingecko.com/api/v3/exchanges/list"
response = requests.get(exchanges_url)
exchanges_list = [exchange['id'] for exchange in response.json()]

# Получаем объемы торгов на одной из бирж за последний день
exchange_id = 'binance'
volume_url = f"https://api.coingecko.com/api/v3/exchanges/{exchange_id}/volume_chart?days=1"
response = requests.get(volume_url)
volume_one_day_data = response.json() 

# Выводим результаты
print("Список всех бирж:", exchanges_list)
print("Объемы торгов на бирже", exchange_id, "за последний день:", volume_one_day_data)

Список всех бирж: ['1bch', '3xcalibur', 'aave', 'abcc', 'acala_swap', 'acdx', 'ace', 'acsi_finance', 'agora_swap', 'alexgo', 'algebra_finance', 'alpha_five', 'altcointrader', 'alterdice', 'altmarkets', 'amaterasu', 'apeswap_bsc', 'apeswap_polygon', 'apeswap_telos', 'apex_pro', 'aprobit', 'aqx', 'aqx_derivatives', 'arbswap_arbitrum_one', 'arthswap', 'astroport', 'auroraswap', 'autoshark_finance', 'aux', 'azbit', 'b2bx', 'babydogeswap', 'babyswap', 'baguette', 'bakeryswap', 'bakkt', 'balanced_network', 'balancer', 'balancer_arbitrum', 'balancer_polygon', 'balancer_v1', 'bancor', 'bancor_v3', 'baryon_network', 'basefex', 'beamswap', 'beaxy', 'beethovenx', 'beethoven_x_optimism', 'benswap_smart_bitcoin_cash', 'bgogo', 'bibox', 'bibox_futures', 'biconomy', 'bigone', 'bigone_futures', 'bilaxy', 'binance', 'binance_dex', 'binance_dex_mini', 'binance_futures', 'binance_us', 'bingx', 'bingx_futures', 'bione', 'birake', 'bisq', 'biswap', 'bit2c', 'bitazza', 'bitbank', 'bitbay', 'bitbegin', 'bitb

Реализуйте генератор — функцию, в которой в цикле: 
- совершается проход по списку всех бирж (он может быть заранее получен из API и сохранён в памяти);
- для каждой биржи выгружается объём торгов за последний день;
- собирается json вида {‘exchange_name’: volume_one_day_data}, чтобы его можно было кодировать для передачи в Kafka (метод str.encode);
- в конце итерации выполняется конструкция yield с кодированным json в виде bytes.

In [242]:
def exchange_volume_generator():

    exchanges = requests.get('https://api.coingecko.com/api/v3/exchanges').json()

    for exchange in exchanges:
        # Получаем идентификатор биржи
        exchange_id = exchange['id']
        exchange_name = exchange['name']

        # Получаем объемы торгов за последний день
        url = f'https://api.coingecko.com/api/v3/exchanges/{exchange_id}/volume_chart?days=1'
        response = requests.get(url)
        try:
            data = response.json()
        except json.JSONDecodeError:
            print(f"Invalid JSON format for {exchange_id} exchange")
            continue
        # Собираем данные в json формате
        one_day_volume_data = []
        for i in data:
            volume_data = i[-1]
            one_day_volume_data.append(volume_data)
        json_data = {exchange_name: one_day_volume_data}
        print(f'Биржа-{exchange_name}:')

        # Кодируем json данные для передачи в Kafka
        encoded_data = json.dumps(json_data).encode('utf-8')

        # Возвращаем данные через yield
        yield encoded_data
        # добавляем небольшую задержку, чтобы не превысить лимит запросов в секунду
        sleep(5)


Реализуйте отправку на сервер Kafka этих данных в ipynb-ноутбуке с продьюсером.

In [243]:
# Создание Producer
config = {'bootstrap.servers': 'localhost:9092', 
          'client.id': socket.gethostname()}

producer = Producer(config)

for data in exchange_volume_generator():
    if data is not None:
        producer.produce(
            topic='quickstart', 
            value=data,
            partition=0,
        )
        print('Сообщение успешно отправлено')

producer.flush()

Биржа-Binance:
Сообщение успешно отправлено
Биржа-Coinbase Exchange:
Сообщение успешно отправлено
Биржа-OKX:
Сообщение успешно отправлено
Биржа-KuCoin:
Сообщение успешно отправлено
Биржа-Bybit:
Сообщение успешно отправлено
Биржа-Huobi:
Сообщение успешно отправлено
Биржа-Binance US:
Сообщение успешно отправлено
Биржа-Bitfinex:
Сообщение успешно отправлено
Биржа-MEXC Global:
Сообщение успешно отправлено
Биржа-Kraken:
Сообщение успешно отправлено
Биржа-Bitget:
Сообщение успешно отправлено
Биржа-Crypto.com Exchange:
Сообщение успешно отправлено
Биржа-Phemex:
Сообщение успешно отправлено
Биржа-Gemini:
Сообщение успешно отправлено
Биржа-BitMart:
Сообщение успешно отправлено
Биржа-Upbit:
Сообщение успешно отправлено
Биржа-XT.COM:
Сообщение успешно отправлено
Биржа-BKEX:
Сообщение успешно отправлено
Биржа-Gate.io:
Сообщение успешно отправлено
Биржа-BTCEX:
Сообщение успешно отправлено
Биржа-BingX:
Сообщение успешно отправлено
Биржа-WhiteBIT:
Сообщение успешно отправлено
Биржа-Bitstamp:
Сообщени

0

In [244]:
producer.flush()


0